In [32]:
import pandas as pd
import os
import numpy as np

In [33]:
df_commune = pd.read_csv('./data/data_commune.csv',usecols=['CODREG','CODDEP','CODCOM','PTOT'] , dtype={'CODREG':'int','CODDEP':'str','CODCOM':'str','PTOT':'int'})

df_ref_geo = pd.read_csv('./data/ref_geo.csv', usecols=['reg_nom','dep_nom','com_code','com_nom_maj'], low_memory=False, dtype={'reg_nom':'str','dep_nom':'str','com_code':'str','com_nom_maj':'str'})

df_valeur = pd.read_csv('./data/val_fonc.csv', 
usecols=['Date mutation','Valeur fonciere', 'No voie','Type de voie','Voie','B/T/Q','Code departement','Code commune','Type local','Surface reelle bati','Nombre pieces principales'], 
dtype={'No voie':'Int64','Code departement':'str','Code commune':'str'}, 
parse_dates=['Date mutation'],
decimal =',',
low_memory=False)

In [34]:
df_commune.head()

,CODREG,CODDEP,CODCOM,PTOT
0,84,01,001,798
1,84,01,002,257
2,84,01,004,14514
3,84,01,005,1776
4,84,01,006,118


In [35]:
df_ref_geo.head()

,reg_nom,dep_nom,com_code,com_nom_maj
0,Auvergne-Rhône-Alpes,Ain,01001,L'ABERGEMENT-CLEMENCIAT
1,Auvergne-Rhône-Alpes,Ain,01002,L'ABERGEMENT-DE-VAREY
2,Auvergne-Rhône-Alpes,Ain,01003,AMAREINS
3,Auvergne-Rhône-Alpes,Ain,01004,AMBERIEU-EN-BUGEY
4,Auvergne-Rhône-Alpes,Ain,01005,AMBERIEUX-EN-DOMBES


In [36]:
df_valeur.head()

,Date mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code departement,Code commune,Type local,Surface reelle bati,Nombre pieces principales
0,2020-01-02,165000.0,347,NaN,RUE,DU CHATEAU,01,103,Appartement,48,3
1,2020-01-02,355680.0,4,NaN,BD,EDOUARD BAUDOIN,06,004,Appartement,40,1
2,2020-01-02,229500.0,20,B,RUE,MARCEAU,06,088,Appartement,82,3
3,2020-01-02,125000.0,550,NaN,RTE,DES VESPINS RN7,06,123,Appartement,27,1
4,2020-01-02,90000.0,9300,NaN,RES,LES ARPEGES BD DES ABA,13,005,Appartement,47,2


### Création de la table tbl_commune

In [37]:
df_commune['id_commune'] = df_commune['CODDEP'] + df_commune['CODCOM']

In [38]:
df_commune.head()

,CODREG,CODDEP,CODCOM,PTOT,id_commune
0,84,01,001,798,01001
1,84,01,002,257,01002
2,84,01,004,14514,01004
3,84,01,005,1776,01005
4,84,01,006,118,01006


In [39]:
df_tbl_commune = pd.merge(df_commune, df_ref_geo, left_on='id_commune',right_on='com_code')

df_tbl_commune = df_tbl_commune.drop(columns='com_code')

df_tbl_commune = df_tbl_commune.rename(columns={'CODREG':'code_reg','CODDEP':'code_dep','CODCOM':'code_com','PTOT':'population','reg_nom':'nom_reg','dep_nom':'nom_dep','com_nom_maj':'nom_commune'})

In [40]:
df_tbl_commune.head()

,code_reg,code_dep,code_com,population,id_commune,nom_reg,nom_dep,nom_commune
0,84,01,001,798,01001,Auvergne-Rhône-Alpes,Ain,L'ABERGEMENT-CLEMENCIAT
1,84,01,002,257,01002,Auvergne-Rhône-Alpes,Ain,L'ABERGEMENT-DE-VAREY
2,84,01,004,14514,01004,Auvergne-Rhône-Alpes,Ain,AMBERIEU-EN-BUGEY
3,84,01,005,1776,01005,Auvergne-Rhône-Alpes,Ain,AMBERIEUX-EN-DOMBES
4,84,01,006,118,01006,Auvergne-Rhône-Alpes,Ain,AMBLEON


In [41]:
df_tbl_commune = df_tbl_commune[['id_commune','code_reg','code_dep','code_com','nom_commune','population','nom_reg','nom_dep']]

In [42]:
df_tbl_commune.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34991 entries, 0 to 34990
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_commune   34991 non-null  object
 1   code_reg     34991 non-null  int64 
 2   code_dep     34991 non-null  object
 3   code_com     34991 non-null  object
 4   nom_commune  34991 non-null  object
 5   population   34991 non-null  int64 
 6   nom_reg      34991 non-null  object
 7   nom_dep      34991 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.1+ MB


In [43]:
df_tbl_commune.to_csv('tbl_commune.csv', index=False, encoding='utf-8')

### Création de la table tbl_bien

In [44]:
df_valeur['id_commune'] = df_valeur['Code departement'] + df_valeur['Code commune']

In [45]:
df_valeur['id_bien'] = np.arange(len(df_valeur))

In [46]:
df_tbl_bien = df_valeur[['id_bien','No voie','Type de voie','Voie','B/T/Q','Type local','Nombre pieces principales', 'Surface reelle bati','id_commune']]

In [47]:
df_tbl_bien = df_tbl_bien.rename(columns={'No voie':'no_voie',
                                  'Type de voie':'type_voie',
                                  'Voie':'voie',
                                  'B/T/Q':'b_t_q',
                                  'Type local':'type_bien',
                                  'Nombre pieces principales':'nb_piece',
                                  'Surface reelle bati':'surface'})

In [48]:
df_tbl_bien.head()

,id_bien,no_voie,type_voie,voie,b_t_q,type_bien,nb_piece,surface,id_commune
0,0,347,RUE,DU CHATEAU,NaN,Appartement,3,48,01103
1,1,4,BD,EDOUARD BAUDOIN,NaN,Appartement,1,40,06004
2,2,20,RUE,MARCEAU,B,Appartement,3,82,06088
3,3,550,RTE,DES VESPINS RN7,NaN,Appartement,1,27,06123
4,4,9300,RES,LES ARPEGES BD DES ABA,NaN,Appartement,2,47,13005


In [49]:
df_tbl_bien.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34169 entries, 0 to 34168
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_bien     34169 non-null  int64 
 1   no_voie     34036 non-null  Int64 
 2   type_voie   33229 non-null  object
 3   voie        34169 non-null  object
 4   b_t_q       2174 non-null   object
 5   type_bien   34169 non-null  object
 6   nb_piece    34169 non-null  int64 
 7   surface     34169 non-null  int64 
 8   id_commune  34169 non-null  object
dtypes: Int64(1), int64(3), object(5)
memory usage: 2.4+ MB


In [50]:
df_tbl_bien.to_csv('tbl_bien.csv', index=False, encoding='utf-8')

### Création de la table tbl_vente

In [51]:
df_tbl_vente = df_valeur[['Date mutation','Valeur fonciere','id_bien']]

In [52]:
df_tbl_vente['id_vente'] = np.arange(len(df_tbl_vente))

/tmp/ipykernel_898/1766589849.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tbl_vente['id_vente'] = np.arange(len(df_tbl_vente))


In [53]:
df_tbl_vente = df_tbl_vente.rename(columns={'Date mutation':'date','Valeur fonciere':'valeur'})

In [54]:
df_tbl_vente = df_tbl_vente[['id_vente','date','valeur','id_bien']]

In [55]:
df_tbl_vente.head()

,id_vente,date,valeur,id_bien
0,0,2020-01-02,165000.0,0
1,1,2020-01-02,355680.0,1
2,2,2020-01-02,229500.0,2
3,3,2020-01-02,125000.0,3
4,4,2020-01-02,90000.0,4


In [56]:
df_tbl_vente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34169 entries, 0 to 34168
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id_vente  34169 non-null  int64         
 1   date      34169 non-null  datetime64[ns]
 2   valeur    34151 non-null  float64       
 3   id_bien   34169 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 1.0 MB


In [57]:
df_tbl_vente = df_tbl_vente.dropna()

In [58]:
df_tbl_vente.to_csv('tbl_vente.csv', index=False, encoding='utf-8')

In [59]:
import sqlite3

# Connexion
conn = sqlite3.connect("LaplaceImmo.db")

# Requête
query = """
SELECT c.nom_reg AS region, COUNT(*) AS nb_appartements_vendus
FROM tbl_vente v
JOIN tbl_bien b ON v.id_bien = b.id_bien
JOIN tbl_commune c ON b.id_commune = c.id_commune
WHERE b.type_bien = 'Appartement'
  AND v.date <= '2020-06-30'
GROUP BY c.nom_reg
ORDER BY nb_appartements_vendus DESC;
"""

# Charger le résultat dans un DataFrame
df = pd.read_sql_query(query, conn)

# Afficher le DataFrame
print(df)

# Fermer
conn.close()

                        region  nb_appartements_vendus
0                Ile-de-France                   13995
1   Provence-Alpes-Côte d'Azur                    3649
2         Auvergne-Rhône-Alpes                    3253
3           Nouvelle-Aquitaine                    1932
4                    Occitanie                    1640
5             Pays de la Loire                    1357
6              Hauts-de-France                    1254
7                    Grand Est                     984
8                     Bretagne                     983
9                    Normandie                     862
10         Centre-Val de Loire                     696
11     Bourgogne-Franche-Comté                     376
12                       Corse                     223
13                  Martinique                      94
14                  La Réunion                      44
15                      Guyane                      34
16                  Guadeloupe                       2


In [60]:
df

,region,nb_appartements_vendus
0,Ile-de-France,13995
1,Provence-Alpes-Côte d'Azur,3649
2,Auvergne-Rhône-Alpes,3253
3,Nouvelle-Aquitaine,1932
4,Occitanie,1640
5,Pays de la Loire,1357
6,Hauts-de-France,1254
7,Grand Est,984
8,Bretagne,983
9,Normandie,862
